In [1]:
from mido import MidiFile
mid = MidiFile('p20190612_midi_test_mix.mid')
mid

<midi file 'p20190612_midi_test_mix.mid' type 1, 1 tracks, 39 messages>

In [2]:
mid.tracks

[<midi track 'Electric Guitar' 39 messages>]

In [92]:
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    a = 0
    midiArray= []
    dicCurrentNote = {}
    currentTime = 0
    for msg in track:
#         print(a)
#         a+=1
#         print(msg)
        if msg.type[:4] == "note":
            currentTime = currentTime + msg.time
            if msg.type == "note_on":
                midiArray.append([msg.note, currentTime, 'OffsetNotYetGiven', 'CatNotYetGiven', 'eventNotGiven'])
                dicCurrentNote[msg.note] = len(midiArray)-1 # save this number of row
            elif msg.type == "note_off":
                midiArray[dicCurrentNote[msg.note]][2] = currentTime # change thie offsetTime by its onset row
#             print(msg)
# print(dicCurrentNote)
# print(midiArray)
midiArray.append([0, currentTime+999, currentTime+1000, 'dummy line', 'dummy line'])
midiArray

Track 0: Electric Guitar


[[45, 0, 240, 'CatNotYetGiven', 'eventNotGiven'],
 [53, 240, 480, 'CatNotYetGiven', 'eventNotGiven'],
 [57, 480, 1440, 'CatNotYetGiven', 'eventNotGiven'],
 [57, 1920, 2400, 'CatNotYetGiven', 'eventNotGiven'],
 [40, 1920, 3840, 'CatNotYetGiven', 'eventNotGiven'],
 [62, 2400, 2880, 'CatNotYetGiven', 'eventNotGiven'],
 [60, 2880, 3360, 'CatNotYetGiven', 'eventNotGiven'],
 [55, 3360, 3840, 'CatNotYetGiven', 'eventNotGiven'],
 [48, 3840, 4800, 'CatNotYetGiven', 'eventNotGiven'],
 [52, 3840, 4800, 'CatNotYetGiven', 'eventNotGiven'],
 [60, 3840, 4800, 'CatNotYetGiven', 'eventNotGiven'],
 [55, 3840, 4800, 'CatNotYetGiven', 'eventNotGiven'],
 [64, 3840, 4800, 'CatNotYetGiven', 'eventNotGiven'],
 [50, 4800, 5760, 'CatNotYetGiven', 'eventNotGiven'],
 [57, 4800, 5760, 'CatNotYetGiven', 'eventNotGiven'],
 [62, 4800, 5760, 'CatNotYetGiven', 'eventNotGiven'],
 [66, 4800, 5760, 'CatNotYetGiven', 'eventNotGiven'],
 [0, 6759, 6760, 'dummy line', 'dummy line']]

In [39]:
""" A function to add count to a dictionary
"""
def dicAddCount(event, dic):
    if event in dic:
        dic[event] = dic[event] + 1
    else:
        dic[event] = 1

In [95]:
rowCount = 0
lenArray = len(midiArray)
eventNumber = 0
dicEvent = {}
while rowCount in range(lenArray):
    
    # process the 1st event
    midiArray[rowCount][4] = eventNumber
    dicAddCount(eventNumber, dicEvent) # count the event number
    start = midiArray[rowCount][1] # assign event start time
    end   = midiArray[rowCount][2] # assign event end time
    
    # initialize for sweeping following notes
    rowCount = rowCount + 1
    chordFlag = 1 # initialize chordFlag
    contEventCount = 1
    
    # sweeping following notes
    while rowCount in range( lenArray ):
        
        # check the following notes are overlap or not
        if midiArray[rowCount][1] < end: 
            midiArray[rowCount][4] = eventNumber # categorized as same event
            dicAddCount(eventNumber, dicEvent) # FUNC: count the event number
            contEventCount = contEventCount + 1 # cumulative count the continuous event
            
            # if one of the following notes have different start or end time,
            # this event is no longer a CHORD anymore.
            # --> disable chord flag
            if (midiArray[rowCount][1] - start != 0) or (midiArray[rowCount][2] - end != 0):
                chordFlag = 0
                
            # update event end time, see the overall-overlapped range
            end = max(end, midiArray[rowCount][2])
            rowCount = rowCount + 1 # step foward
            
        # if following note isn't overlapped, call it a new event
        else: 
            eventNumber = eventNumber + 1 # next event
            
            # determine category of previous event
            if contEventCount == 1:
                midiArray[rowCount -1 ][3] = 'MEL' # melody
            elif chordFlag == 1:
                for row in range(rowCount -contEventCount , rowCount):
                    midiArray[row][3] = 'CHO' # chord
            else:
                for row in range(rowCount -contEventCount , rowCount):
                    midiArray[row][3] = 'MIX' # mix = chord + melody
            break # break while

midiArray

[[45, 0, 240, 'MEL', 0],
 [53, 240, 480, 'MEL', 1],
 [57, 480, 1440, 'MEL', 2],
 [57, 1920, 2400, 'MIX', 3],
 [40, 1920, 3840, 'MIX', 3],
 [62, 2400, 2880, 'MIX', 3],
 [60, 2880, 3360, 'MIX', 3],
 [55, 3360, 3840, 'MIX', 3],
 [48, 3840, 4800, 'CHO', 4],
 [52, 3840, 4800, 'CHO', 4],
 [60, 3840, 4800, 'CHO', 4],
 [55, 3840, 4800, 'CHO', 4],
 [64, 3840, 4800, 'CHO', 4],
 [50, 4800, 5760, 'CHO', 5],
 [57, 4800, 5760, 'CHO', 5],
 [62, 4800, 5760, 'CHO', 5],
 [66, 5800, 5900, 'MEL', 6],
 [0, 6759, 6760, 'dummy line', 7]]

In [94]:
midiArray[16][1] = 5800
midiArray[16][2] = 5900
lenArray

18

In [37]:
eventDic = {}
for i in range(lenArray):
    if midiArray[i][4] in eventDic:
        eventDic[midiArray[i][4]] = eventDic[midiArray[i][4]] + 1
    else:
        eventDic[midiArray[i][4]] = 1
eventDic

{0: 1, 1: 1, 2: 1, 3: 5, 4: 5, 5: 4}

In [5]:
print(track[6].type)
print(track[6].note)
print(track[6].time)

note_on
53
0


In [6]:
import numpy as np
a = np.empty((0,4))
a = np.append(a,[[1,2,3,4]])

a = np.append([a], [[5,6,7,8]],axis=0)
print(a)
a[1,1] = 7.6
# a.shape
print(a)

[[1. 2. 3. 4.]
 [5. 6. 7. 8.]]
[[1.  2.  3.  4. ]
 [5.  7.6 7.  8. ]]


In [62]:
b = []
b.append([1,2,3,4])
b.append([5,6,7,8])
# b[0][0] + b[0][1]
b[1].append(2)
b
for row in b:
    row[1] = 1
b

[[1, 1, 3, 4], [5, 1, 7, 8, 2]]

In [36]:
dicCurrentNote

{45: 0,
 53: 1,
 57: 14,
 40: 4,
 62: 15,
 60: 10,
 55: 11,
 48: 8,
 52: 9,
 64: 12,
 50: 13,
 66: 16}